# Integrator  

Idea is that this will show how we can approximate a definite integral. It's actually very interesting and you'll have fun reading this.  

#### Basic Integration

Let's start with: $$\int_{0}^{1} x^2\,dx$$.  
Since we are calculating the area under the graph in the interval $[0,1]$.  
If you notice, this is actually the same as calculating ratio of circle_area and square_area, like we did in **monte_carlo_pi_estimation.ipynb**  

Now, what we will do is follow the method of Riemann Sum to approximate the solution:  
1. Take random uniform values in the interval $[0,1]$ - we will take a number of sample points.
2. Calculate $f(x) = x^2$.
3. Sum all those values together.
4. Divide the sum obtained by total number of sample points chosen.

It is the Law of Large Number that ensures our approximation will converge to the actual value as sample size increases.  


In [8]:
import random
import numpy as np

In [16]:
def x_squared_function(n):
    total_sum = 0.0
    for _ in range(n):
        x = random.uniform(0,1)
        total_sum += x**2
    return total_sum / n

# For sin(x)dx in the interval [0,1]
def sin_function(n):
    total_sum = 0.0
    for _ in range(n):
        x = random.uniform(0,1)
        total_sum += np.sin(x)
    return total_sum / n

sample_test = [10, 100, 1000, 10000, 100000]
print("For x^2, Actual Value: 0.33...")
for n in sample_test:
    print(f"{x_squared_function(n):.5f}") 

print("\nFor sin(x), actual value approx: 0.45969...")
for n in sample_test:
    print(f"{sin_function(n):.5f}")

For x^2, Actual Value: 0.33...
0.25315
0.34867
0.33492
0.33573
0.33344

For sin(x), actual value approx: 0.45969...
0.43175
0.49833
0.45243
0.46015
0.45965


#### Based on the above, this is a monte carlo based Integrator that can approximate the definite integration  

In [34]:
# General Integrator in the interval [a, b]
def monte_carlo_integrator(f, a, b, n):
    total_sum = 0.0
    for _ in range(n):
        x = random.uniform(a,b) # sampling in the interval [a,b]
        total_sum += f(x)
    return total_sum/n

# print(monte_carlo_integrator(lambda x: x**2, 0, 1, 100000)) # works correctly

def print_values(f, a, b, n):
    print(f"For function {f}: ")
    sample_test = [10, 100, 1000, 10000, 100000]
    for n in sample_test:
        print(f"{monte_carlo_integrator(f, a, b, n):.5f}")
    print("\n")

# Gives good result. If I can somehow just define a function and
# call it as parameter to print_values - it will work smoothly
print_values(lambda x: np.sin(x), 0, 1, 100000)

For function <function <lambda> at 0x73ef2953fb50>: 
0.35583
0.39771
0.46370
0.46240
0.46067




#### Error checking: Shows our estimate based on a confidence Intervals

Will keep it basic and direct this time. 

I want to see how much is my estimate jumping around. Is there a fixed range? Can I decrease that range for better estimate?

Answer:
Will use confidence intervals and idea of mean (M) and variance (V).  
Standard Error (SE) is based on standard variance and number of samples - will use it to get our confidence Interval.  

Then our estimte will be as M +- SE

In [38]:
import random
import math

def monte_carlo_integrator_with_error(f, n):
    samples = [f(random.uniform(0,1)) for _ in range(n)]
    mean = sum(samples) / n
    variance = sum((x - mean)**2 for x in samples)/ (n-1)
    standard_error = math.sqrt(variance) / math.sqrt(n)
    return mean, standard_error

In [44]:
estimate, error = monte_carlo_integrator_with_error(math.sin, 10000)
print(f"Estimate: {estimate:5f} +- {error:5f}")

Estimate: 0.457696 +- 0.002471


#### Importance Sampling Idea

Starting with obserbing that the uniform sampling does not work properly on the function $$\int_{0}^{1} \frac{1}{1 + x^2}\,dx = \tan^{-1}(1) = \frac{\pi}{4} \approx 0.7854 $$.

In [48]:
estimate, error = monte_carlo_integrator_with_error(lambda x: 1 / (1 + x**2), 10000)
print(f"Estimate: {estimate:5f} +- {error:5f}")

Estimate: 0.786076 +- 0.001609


Answer is almost correct, but **Important Sampling** raises the main questions:  
Can we get the same accuracy with fewer samples, or less error for the same effort?

That’s where Importance Sampling steps in. Will study on next notebook